In [1]:

import pandas as pd
import math
import numpy as np

data = pd.read_csv("/Users/jyothsnadevitiruveedhula/Documents/Train_ML_Lab_3/Sheet 2-Training1.csv")
features = [feat for feat in data]
features.remove("Parents")
features

['Weather', 'Money', 'Decision(Category)']

In [2]:
#is called every time an object is created from a class-->initialising
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [3]:
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Parents"] == "Yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

In [4]:
def info_gain(examples, attr):
    uniq = np.unique(examples[attr])
    print ("\n",uniq)
    gain = entropy(examples)
    print ("\n",gain)
    for u in uniq:
        subdata = examples[examples[attr] == u]
        print ("\n",subdata)
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
    print ("\n",gain)
    return gain

In [5]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
        #print ("\n",examples)
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    #print ("\nMax feature attr",max_feat)
    uniq = np.unique(examples[max_feat])
    #print ("\n",uniq)
    for u in uniq:
        #print ("\n",u)
        subdata = examples[examples[max_feat] == u]
        #print ("\n",subdata)
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Parents"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [6]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [7]:
root = ID3(data, features)
def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)


 ['Rainy' 'Sunny' 'Windy']

 1.0

   Weather Money Decision(Category) Parents
3   Rainy  Poor             Cinema     Yes
4   Rainy  Rich            Stay in      No
5   Rainy  Poor             Cinema     Yes

   Weather Money Decision(Category) Parents
0   Sunny  Rich             Cinema     Yes
1   Sunny  Rich             Tennis      No
9   Sunny  Rich             Tennis      No

   Weather Money Decision(Category) Parents
2   Windy  Rich             Cinema     Yes
6   Windy  Poor             Cinema      No
7   Windy  Rich           Shopping      No
8   Windy  Rich             Cinema     Yes

 0.049022499567306366

 ['Poor' 'Rich']

 1.0

   Weather Money Decision(Category) Parents
3   Rainy  Poor             Cinema     Yes
5   Rainy  Poor             Cinema     Yes
6   Windy  Poor             Cinema      No

   Weather Money Decision(Category) Parents
0   Sunny  Rich             Cinema     Yes
1   Sunny  Rich             Tennis      No
2   Windy  Rich             Cinema     Yes
4   Ra

In [8]:

print("Decision Tree is:")
printTree(root)

new = {"Weekend":"W1","Weather":"Rainy", "Money":"poor", "Decision(Category)":"Cinema"}
classify (root, new)

Decision Tree is:
Decision(Category)
	Cinema
		Weather
			Rainy ->  ['Yes']

			Sunny ->  ['Yes']

			Windy
				Money
					Poor ->  ['No']

					Rich ->  ['Yes']

	Shopping ->  ['No']

	Stay in ->  ['No']

	Tennis ->  ['No']

Predicted Label for new example {'Weekend': 'W1', 'Weather': 'Rainy', 'Money': 'poor', 'Decision(Category)': 'Cinema'}  is: ['Yes']
